In [1]:
import json 
import pandas as pd

In [48]:
with open("out.json", 'r') as fdata:
    data = json.load(fdata)

In [49]:
df = pd.DataFrame(data)
df

,har_0,har_1,har_2,har_3,har_4,har_5,har_6,har_7,har_8,har_9,...,bank_0,bank_1,bank_2,bank_3,bank_4,bank_5,bank_6,bank_7,bank_8,bank_9
MDS_raw,3.857761e+05,3.899252e+05,3.989198e+05,3.904292e+05,3.985287e+05,3.889685e+05,3.948347e+05,3.874500e+05,3.862357e+05,3.884417e+05,...,3.340875e+06,3.309720e+06,3.297608e+06,3.317489e+06,3.286458e+06,3.311263e+06,3.312068e+06,3.317415e+06,3.299065e+06,3.316334e+06
MDS_norm,1.892851e-01,1.903003e-01,1.924827e-01,1.904233e-01,1.923883e-01,1.900667e-01,1.914946e-01,1.896953e-01,1.893978e-01,1.899380e-01,...,3.577325e-01,3.560605e-01,3.554084e-01,3.564782e-01,3.548071e-01,3.561435e-01,3.561868e-01,3.564742e-01,3.554869e-01,3.564161e-01
MDS_scalenorm,1.892851e-01,1.903003e-01,1.924827e-01,1.904233e-01,1.923883e-01,1.900667e-01,1.914946e-01,1.896953e-01,1.893978e-01,1.899380e-01,...,3.577325e-01,3.560605e-01,3.554084e-01,3.564782e-01,3.548071e-01,3.561435e-01,3.561868e-01,3.564742e-01,3.554869e-01,3.564161e-01
MDS_kruskal,1.459952e-01,1.473284e-01,1.497061e-01,1.475579e-01,1.494295e-01,1.472933e-01,1.487018e-01,1.462131e-01,1.458909e-01,1.472100e-01,...,3.201178e-01,3.158464e-01,3.151969e-01,3.174094e-01,3.124974e-01,3.162028e-01,3.156323e-01,3.167153e-01,3.143835e-01,3.168455e-01
MDS_sheppard,9.477211e-01,9.454437e-01,9.433002e-01,9.455008e-01,9.435913e-01,9.456057e-01,9.441489e-01,9.467062e-01,9.479136e-01,9.463944e-01,...,6.243065e-01,6.367410e-01,6.407572e-01,6.335615e-01,6.447443e-01,6.357317e-01,6.358585e-01,6.337724e-01,6.408379e-01,6.333379e-01
TSNE_raw,4.129114e+08,3.806237e+08,4.137609e+08,3.645546e+08,3.481111e+08,3.816695e+08,4.012655e+08,2.939056e+08,3.330274e+08,3.892305e+08,...,6.460869e+09,5.853206e+09,6.319561e+09,6.611213e+09,6.597762e+09,6.186895e+09,5.521767e+09,5.315046e+09,5.791994e+09,5.036879e+09
TSNE_norm,6.192660e+00,5.945615e+00,6.199028e+00,5.818756e+00,5.686012e+00,5.953777e+00,6.104706e+00,5.224597e+00,5.561460e+00,6.012461e+00,...,1.573163e+01,1.497356e+01,1.555864e+01,1.591361e+01,1.589742e+01,1.539447e+01,1.454344e+01,1.426861e+01,1.489506e+01,1.389021e+01
TSNE_scalenorm,2.726790e-01,2.629721e-01,2.699534e-01,2.612807e-01,2.699339e-01,2.788020e-01,2.697914e-01,2.651101e-01,2.701522e-01,2.643120e-01,...,4.441063e-01,4.365312e-01,4.285409e-01,4.434325e-01,4.441962e-01,4.413623e-01,4.246376e-01,4.228714e-01,4.437075e-01,4.338715e-01
TSNE_kruskal,2.390122e-01,2.282536e-01,2.280894e-01,2.291657e-01,2.433236e-01,2.438441e-01,2.365527e-01,2.450593e-01,2.444778e-01,2.268770e-01,...,4.091810e-01,4.033117e-01,3.945089e-01,4.074875e-01,4.106019e-01,4.107751e-01,3.911568e-01,3.886872e-01,4.081661e-01,3.984971e-01
TSNE_sheppard,8.686662e-01,8.769171e-01,8.782712e-01,8.757794e-01,8.641604e-01,8.515719e-01,8.719078e-01,8.567790e-01,8.615252e-01,8.796681e-01,...,5.219340e-01,5.206613e-01,5.320773e-01,5.282052e-01,5.147583e-01,5.112636e-01,5.300061e-01,5.393710e-01,5.271898e-01,5.297768e-01


In [50]:
data = df.transpose()
data = data.drop([col for col in data.columns if "UMAP" in col],axis=1)

In [70]:
metric_names = ["_raw", "_norm", "_scalenorm", "_kruskal", "sheppard"]

In [71]:
res = dict()
for mname in metric_names: 
    metric = data[(col for col in data.columns if mname in col)]
    name_lookup = ["MDS", "TSNE", "RANDOM"]
    orderCount = {
        ("MDS", "TSNE", "RANDOM"): 0,
        ("MDS", "RANDOM", "TSNE"): 0,
        ("TSNE", "MDS", "RANDOM"): 0,
        ("TSNE", "RANDOM", "MDS"): 0,
        ("RANDOM", "MDS", "TSNE"): 0,
        ("RANDOM", "TSNE", "MDS"): 0,                    
    }

    for ind,row in metric.iterrows():
        tup_row = sorted([(row[i], name_lookup[i]) for i in range(3)])
        order = tuple([tup[1] for tup in tup_row])
        orderCount[order] += 1

    totalvals = sum(orderCount.values())
    for key in orderCount.keys():
        orderCount[key] /= totalvals
        
    res[mname] = orderCount

In [77]:
res

{'_raw': {('MDS', 'TSNE', 'RANDOM'): 0.041666666666666664,
  ('MDS', 'RANDOM', 'TSNE'): 0.8333333333333334,
  ('TSNE', 'MDS', 'RANDOM'): 0.125,
  ('TSNE', 'RANDOM', 'MDS'): 0.0,
  ('RANDOM', 'MDS', 'TSNE'): 0.0,
  ('RANDOM', 'TSNE', 'MDS'): 0.0},
 '_norm': {('MDS', 'TSNE', 'RANDOM'): 0.041666666666666664,
  ('MDS', 'RANDOM', 'TSNE'): 0.8333333333333334,
  ('TSNE', 'MDS', 'RANDOM'): 0.125,
  ('TSNE', 'RANDOM', 'MDS'): 0.0,
  ('RANDOM', 'MDS', 'TSNE'): 0.0,
  ('RANDOM', 'TSNE', 'MDS'): 0.0},
 '_scalenorm': {('MDS', 'TSNE', 'RANDOM'): 0.9083333333333333,
  ('MDS', 'RANDOM', 'TSNE'): 0.08333333333333333,
  ('TSNE', 'MDS', 'RANDOM'): 0.008333333333333333,
  ('TSNE', 'RANDOM', 'MDS'): 0.0,
  ('RANDOM', 'MDS', 'TSNE'): 0.0,
  ('RANDOM', 'TSNE', 'MDS'): 0.0},
 '_kruskal': {('MDS', 'TSNE', 'RANDOM'): 0.8208333333333333,
  ('MDS', 'RANDOM', 'TSNE'): 0.17916666666666667,
  ('TSNE', 'MDS', 'RANDOM'): 0.0,
  ('TSNE', 'RANDOM', 'MDS'): 0.0,
  ('RANDOM', 'MDS', 'TSNE'): 0.0,
  ('RANDOM', 'TSNE', 'MDS

In [83]:
stab = " & " + " & ".join(mname.replace("_","") for mname in metric_names)
stab += "\\hline \\\\ \n"
stab

' & raw & norm & scalenorm & kruskal & sheppard\\hline \\\\ \n'

In [84]:
for row in res['_raw'].keys():
    stab += "<".join(el for el in row)
    stab += " & "
    for metric in res.keys(): 
        stab += f"{round(res[metric][row], 3)}"
        if metric != "sheppard": 
            stab += " & "
    stab += "\\\\ \n"
    

In [85]:
print(stab)

 & raw & norm & scalenorm & kruskal & sheppard\hline \\ 
MDS<TSNE<RANDOM & 0.042 & 0.042 & 0.908 & 0.821 & 0.0 & \\ 
MDS<RANDOM<TSNE & 0.833 & 0.833 & 0.083 & 0.179 & 0.0 & \\ 
TSNE<MDS<RANDOM & 0.125 & 0.125 & 0.008 & 0.0 & 0.0 & \\ 
TSNE<RANDOM<MDS & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & \\ 
RANDOM<MDS<TSNE & 0.0 & 0.0 & 0.0 & 0.0 & 0.083 & \\ 
RANDOM<TSNE<MDS & 0.0 & 0.0 & 0.0 & 0.0 & 0.917 & \\ 

